In [1]:
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
%%time

df = pd.read_json(r'/opt/models/codefordemocracy/data/immi_articles.json', orient='columns')

CPU times: user 2.68 s, sys: 611 ms, total: 3.3 s
Wall time: 3.55 s


## Basic EDA

In [3]:
df.describe()

,_score
count,50004.000000
mean,5.797664
std,1.180294
min,0.039519
25%,5.183180
50%,5.817132
75%,6.447397
max,10.616549


In [4]:
df.count()

_index     50004
_type      50004
_id        50004
_score     50004
_source    50004
dtype: int64

In [5]:
df.shape

(50004, 5)

In [6]:
#print 1st column 1st row
df.iloc[0] # this is same as df[[0]].values[0][0]

_index                                              articles
_type                                                   _doc
_id                                     smcc4G4BNQspL_riokgN
_score                                               10.6165
_source    {'metadata': {'keywords': [''], 'description':...
Name: 0, dtype: object

In [7]:
df_clean= df.drop(['_id','_score', '_type'], axis=1)

In [8]:
df_clean.shape

(50004, 2)

In [9]:
df_clean.head(5)

,_index,_source
0,articles,"{'metadata': {'keywords': [''], 'description':..."
1,articles,"{'metadata': {'keywords': [''], 'description':..."
2,articles,"{'metadata': {'keywords': [''], 'description':..."
3,articles,"{'metadata': {'keywords': [''], 'description':..."
4,articles,{'url': 'http://thehill.com/opinion/immigratio...


In [10]:
df_clean.iloc[0]

_index                                              articles
_source    {'metadata': {'keywords': [''], 'description':...
Name: 0, dtype: object

In [11]:
#df.iloc[0]

_index                                              articles
_type                                                   _doc
_id                                     smcc4G4BNQspL_riokgN
_score                                               10.6165
_source    {'metadata': {'keywords': [''], 'description':...
Name: 0, dtype: object

In [12]:
#df.describe()

,_score
count,50004.000000
mean,5.797664
std,1.180294
min,0.039519
25%,5.183180
50%,5.817132
75%,6.447397
max,10.616549


In [26]:
df.dtypes

_index     object
_type      object
_id        object
_score      int64
_source    object
dtype: object

In [13]:
#Get value within a Series
df.loc[0].at['_type']

'_doc'

In [14]:
#Get value within a Series
df.loc[0].at['_source']

{'metadata': {'keywords': [''],
  'description': 'What’s the big worry? Immigration, immigration, immigration',
  'language': 'en',
  'dump': {'twitter': {'image': 'https://www.conservativewoman.co.uk/wp-content/uploads/2019/03/immigration-1.jpg',
    'site': '@TheConWom',
    'creator': '@TheConWom',
    'description': 'What’s the big worry? Immigration, immigration, immigration',
    'title': 'What’s the big worry? Immigration, immigration, immigration | The Conservative Woman',
    'card': 'summary'},
   'viewport': 'width=device-width, initial-scale=1.0',
   'og': {'site_name': 'The Conservative Woman',
    'updated_time': '2019-12-06T16:39:09+00:00',
    'image': {'identifier': 'https://www.conservativewoman.co.uk/wp-content/uploads/2019/03/immigration-1.jpg',
     'secure_url': 'https://www.conservativewoman.co.uk/wp-content/uploads/2019/03/immigration-1.jpg',
     'width': 600,
     'height': 300},
    'description': 'What’s the big worry? Immigration, immigration, immigration',

In [15]:
df_src=df.drop(['_index','_id','_score', '_type'], axis=1)

In [16]:
df_src.dtypes

_source    object
dtype: object

In [17]:
%%time

df_src_norm = json_normalize(df_src['_source'])

CPU times: user 41.5 s, sys: 548 ms, total: 42 s
Wall time: 42 s


In [18]:
#list()

[]

In [19]:
df_src_norm.head(3)

,url,metadata.keywords,metadata.description,metadata.language,metadata.dump.twitter.image,metadata.dump.twitter.site,metadata.dump.twitter.creator,metadata.dump.twitter.description,metadata.dump.twitter.title,metadata.dump.twitter.card,...,metadata.dump.citation_journal_abbrev,metadata.dump.citation_pages,metadata.dump.citation_language,metadata.dump.citation_keywords,metadata.dump.citation_date,metadata.dump.citation_author_institution,metadata.dump.specificfeeds-verification-code-Ri9Tb2pQYmlIUzh5M0VBdVovdmZMWitWaUp0NTF4MlpVSzVTMkRKbmRaQ2VzblhpUWdLcmpqM0dWM3pZcGxheEZCU0RCaGNVU28wTTJVRmFLcll6U3FMaTFPcVVhZlorb3B0cHZtYXRlTXA0RDFzNjRZU0N6MjRuSjZ4dGxlOFd8bTllRExVSS9iUDdldEkzOHJlSGYvL3cydTIvb3FhQ2RvaTAxdDczUklJTT0=,metadata.dump.webpack-public-path,metadata.dump.sentry-dsn,metadata.dump.contentUrl
0,https://www.conservativewoman.co.uk/whats-the-...,[],"What’s the big worry? Immigration, immigration...",en,https://www.conservativewoman.co.uk/wp-content...,@TheConWom,@TheConWom,"What’s the big worry? Immigration, immigration...","What’s the big worry? Immigration, immigration...",summary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://gulfnews.com/business/val-morgan-immigr...,[],Val Morgan Immigration specialises in UK immig...,en,NaN,@gulf_news,NaN,Val Morgan Immigration specialises in UK immig...,Val Morgan Immigration Advisers: Secure your f...,summary_large_image,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.immattyllp.com,[],"Immigration Attorneys, LLP - U.S. Immigration ...",en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_src_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50004 entries, 0 to 50003
Columns: 2254 entries, url to metadata.dump.contentUrl
dtypes: bool(2), float64(267), object(1985)
memory usage: 859.2+ MB


In [21]:
df_src_norm.shape

(50004, 2254)

In [22]:
df_src_norm_limited=df_src_norm.filter(items=['url', 'site_name','extracted.title', 'extracted.date', 'extracted.text', 'metadata.keywords', 'metadata.description'])

In [23]:
df_src_norm_limited.shape

(50004, 6)

In [24]:
df_src_norm_limited.head(10)

,url,extracted.title,extracted.date,extracted.text,metadata.keywords,metadata.description
0,https://www.conservativewoman.co.uk/whats-the-...,"What’s the big worry? Immigration, immigration...",2019-12-07T01:15:00+00:00,HOW many times in the past few months have we ...,[],"What’s the big worry? Immigration, immigration..."
1,http://gulfnews.com/business/val-morgan-immigr...,Val Morgan Immigration Advisers: Secure your f...,2019-11-21T00:00:36.623000+04:00,Val Morgan has been an integral part of the UK...,[],Val Morgan Immigration specialises in UK immig...
2,http://www.immattyllp.com,Immigration Attorneys LLP,None,,[],"Immigration Attorneys, LLP - U.S. Immigration ..."
3,http://gulfnews.com/business/top-immigration-c...,Top immigration consultants in UAE: Secure you...,2019-11-21T00:00:36.623000+04:00,Val Morgan has been an integral part of the UK...,[],Val Morgan Immigration specialises in UK immig...
4,http://thehill.com/opinion/immigration/512266-...,The prospect of a Kamala Harris presidency giv...,2020-08-17T11:00:09-04:00,Joe Biden has chosen U.S. Sen. Kamala D. Harri...,"[Illegal immigration to the United States, def...",Harris’s immigration policies would encourage ...
5,http://onenewsnow.com/legal-courts/2020/03/30/...,Safeguards suggested to keep courts moving,2020-03-30T00:00:00,An immigration reform activist and former immi...,[],An immigration reform activist and former immi...
6,http://americanimmigrationcouncil.org/news/new...,New Documents Reveal Immigration Judge Hiring ...,2020-05-01T15:00:00-04:00,"WASHINGTON—Today, the American Immigration Law...",[],The American Immigration Lawyers Association a...
7,http://fairus.org/news/newsletters,Federation for American Immigration Reform,None,Over 1M Illegal Aliens Expected to Arrive in U...,[],FAIR's Immigration Report is the nation's most...
8,http://americanimmigrationcouncil.org/news/com...,Complaint Demands Oversight of Customs and Bor...,2019-09-04T12:54:00-04:00,"WASHINGTON, DC – Today, the American Immigrati...",[],"WASHINGTON, DC – Today, the American Immigrati..."
9,http://ow.ly/kBD22,What immigration reform must entail,None,"Wed May 1, 2013 7:12 AM\n\nAmerica is a nation...","[What immigration reform must entail, Arizona,...","What immigration reform must entail,Arizona co..."


In [25]:
df_src_norm_limited['metadata.description'][0]

'What’s the big worry? Immigration, immigration, immigration'

In [26]:
df_src_norm_limited['extracted.text'][0]

'HOW many times in the past few months have we heard politicians and commentators, and those who benefit from high levels of immigration, say that the public is no longer bothered about the very high levels of immigration that this country continues to experience?\n\nThis is of course untrue and a new Deltapoll, commissioned by Migration Watch UK, gives the lie to the claim (see summary here and report by the Daily Mail here).\n\nNotwithstanding what we have been repeatedly told, the survey has found that nearly two-thirds of respondents (65 per cent) agree that recent levels of overseas net migration to the UK are a source of major concern for the public. Fewer than a quarter (22 per cent) disagree.\n\nSo why has immigration not been given the prominence in the election campaign that the public clearly believe it merits? Admittedly, as the campaign has worn on and the politicians have knocked on doors in search of votes, it has dawned on them that the electorate are not going to be fo

In [27]:
df_src_norm_limited['url'][0]

'https://www.conservativewoman.co.uk/whats-the-big-worry-immigration-immigration-immigration/'

In [28]:
df_src_norm_limited.describe

<bound method NDFrame.describe of                                                      url  \
0      https://www.conservativewoman.co.uk/whats-the-...   
1      http://gulfnews.com/business/val-morgan-immigr...   
2                              http://www.immattyllp.com   
3      http://gulfnews.com/business/top-immigration-c...   
4      http://thehill.com/opinion/immigration/512266-...   
...                                                  ...   
49999  http://www.realclearpolitics.com/video/2019/05...   
50000  http://www.realclearpolitics.com/video/2019/06...   
50001  http://www.realclearpolitics.com/video/2019/11...   
50002  https://www.realclearpolitics.com/video/2017/1...   
50003  http://cfr.org/event/2020-census-risks-getting...   

                                         extracted.title  \
0      What’s the big worry? Immigration, immigration...   
1      Val Morgan Immigration Advisers: Secure your f...   
2                              Immigration Attorneys LLP   
3    

# Topic Modeling

## Using Latent Dirichlet Allocation (LDA)
#### The LDA is based upon two general assumptions:

### Documents that have similar words usually have the same topic
### Documents that have groups of words frequently occurring together usually have the same topic.


# Vocabulary Creation

### Using CountVectorizer class from the sklearn.feature_extraction.text module to create a document-term matrix. We specify to only include those words that appear in less than 80% of the document and appear in at least 2 documents. We also remove all the stop words as they do not really contribute to topic modeling.

In [29]:
extracted_text=df_src_norm_limited['extracted.text']
extracted_text.dropna()

0        HOW many times in the past few months have we ...
1        Val Morgan has been an integral part of the UK...
2                                                         
3        Val Morgan has been an integral part of the UK...
4        Joe Biden has chosen U.S. Sen. Kamala D. Harri...
                               ...                        
49999    Rep. Adam Schiff said Sunday on CBS's "Face th...
50000    Sidney Powell, author of 'Licensed to Lie,' sa...
50001    Trump Delivers Remarks on Federal Judicial Con...
50002    President Donald Trump delivered remarks on ta...
50003    Panelists discuss preparations for the 2020 U....
Name: extracted.text, Length: 50004, dtype: object

In [30]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
#doc_term_matrix = count_vect.fit_transform(df_src_norm_limited['extracted.text'].values.astype('U'))
doc_term_matrix = count_vect.fit_transform(extracted_text.values.astype('U'))


In [31]:
doc_term_matrix

<50004x74205 sparse matrix of type '<class 'numpy.int64'>'
	with 9505526 stored elements in Compressed Sparse Row format>

In [61]:
# Has 9505526 stored elements size vocabulary in this dataset

In [32]:
%%time

# Next, we will use LDA to create topics along with the probability distribution for each word in our vocabulary for each topic. 
# LatentDirichletAllocation class from the sklearn.decomposition library to perform LDA on our document-term matrix. 
# The parameter n_components specifies the number of categories, or topics, that we want our text to be divided into. 
# The parameter random_state (aka the seed) is set to 42
# Number of categories is set by n_components

LDA = LatentDirichletAllocation(n_components=20, random_state=42)
LDA.fit(doc_term_matrix)

CPU times: user 1h 32min 13s, sys: 2min 6s, total: 1h 34min 20s
Wall time: 10min 10s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [33]:
first_topic = LDA.components_[0]

In [34]:
%%time
top_topic_words = first_topic.argsort()[-10:]

CPU times: user 16.8 ms, sys: 20 µs, total: 16.8 ms
Wall time: 14.7 ms


In [35]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

workers
think
work
don
know
going
just
like
people
said


In [36]:
second_topic = LDA.components_[1]

In [37]:
%%time
top_topic_words = second_topic.argsort()[-10:]

CPU times: user 13.4 ms, sys: 128 µs, total: 13.5 ms
Wall time: 10.2 ms


In [38]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

sanctuary
new
local
county
state
said
law
federal
enforcement
ice


In [39]:
%%time
# 10 words with highest probabilities for all the five topics:


for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['workers', 'think', 'work', 'don', 'know', 'going', 'just', 'like', 'people', 'said']


Top 10 words for topic #1:
['sanctuary', 'new', 'local', 'county', 'state', 'said', 'law', 'federal', 'enforcement', 'ice']


Top 10 words for topic #2:
['access', 'news', 'says', 'omar', 'people', 'new', 'com', 'canadian', 'said', 'canada']


Top 10 words for topic #3:
['medical', 'covid', 'facility', '19', 'health', 'center', 'detainees', 'said', 'ice', 'detention']


Top 10 words for topic #4:
['debate', 'campaign', 'obama', 'people', 'democrats', 'democratic', 'biden', 'said', 'president', 'trump']


Top 10 words for topic #5:
['miller', 'posted', 'group', 'like', 'media', 'people', 'new', 'post', 'anti', 'white']


Top 10 words for topic #6:
['uk', 'kong', '2019', 'hong', 'images', 'reuters', '2020', 'afp', '50', 'getty']


Top 10 words for topic #7:
['000', 'people', 'homeland', 'enforcement', 'illegal', 'trump', 'administration', 'said', 'security', 'border']


Top

In [40]:
# we will add a column to the original data frame that will store the topic for the text. 
#To do so, we can use LDA.transform() method and pass it our document-term matrix. 
#This method will assign the probability of all the topics to each document

topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(50004, 20)

In [41]:
%%time
#adds a new column for topic in the data frame and assigns the topic value to each row in the column:
# To find the topic index with maximum value, we can call the argmax() method and pass 1 as the value for the axis parameter.

df_src_norm_limited['Topic'] = topic_values.argmax(axis=1)

CPU times: user 10.1 ms, sys: 50 µs, total: 10.1 ms
Wall time: 8.45 ms


In [42]:
df_src_norm_limited.head()

,url,extracted.title,extracted.date,extracted.text,metadata.keywords,metadata.description,Topic
0,https://www.conservativewoman.co.uk/whats-the-...,"What’s the big worry? Immigration, immigration...",2019-12-07T01:15:00+00:00,HOW many times in the past few months have we ...,[],"What’s the big worry? Immigration, immigration...",12
1,http://gulfnews.com/business/val-morgan-immigr...,Val Morgan Immigration Advisers: Secure your f...,2019-11-21T00:00:36.623000+04:00,Val Morgan has been an integral part of the UK...,[],Val Morgan Immigration specialises in UK immig...,10
2,http://www.immattyllp.com,Immigration Attorneys LLP,None,,[],"Immigration Attorneys, LLP - U.S. Immigration ...",0
3,http://gulfnews.com/business/top-immigration-c...,Top immigration consultants in UAE: Secure you...,2019-11-21T00:00:36.623000+04:00,Val Morgan has been an integral part of the UK...,[],Val Morgan Immigration specialises in UK immig...,10
4,http://thehill.com/opinion/immigration/512266-...,The prospect of a Kamala Harris presidency giv...,2020-08-17T11:00:09-04:00,Joe Biden has chosen U.S. Sen. Kamala D. Harri...,"[Illegal immigration to the United States, def...",Harris’s immigration policies would encourage ...,4


In [43]:
df_src_norm_limited['extracted.text'][1]

'Val Morgan has been an integral part of the UK and Dubai’s immigration services sector for the past decade Image Credit: Supplied\n\nVal Morgan Immigration Advisers specialises is securing your families future.\n\nVal Morgan has been an integral part of the UK and Dubai’s immigration services sector for the past decade.\n\nIt is recognised as a premier legal consultant delivering clients visionary life goals to emigrate to some of the worlds most desired locations including the UK, New Zealand, Australia, Europe and Canada.\n\nVal Morgan Immigration Advisers specialises in UK immigration under the Overseas Business Representative Programme and the newly announced UK Startup and Innovator visa, which requires minimal investments.\n\nAs one of the most renowned international immigration advisers in the region, this year the company became the first law firm in the Middle East and South Asia to receive the Innovator visa approval.\n\nFrom a global perspective, Val Morgan was the fifth la

# Topic Models Using
## Non-Negative Matrix Factorization (NMF)

### Non-negative matrix factorization is also a supervised learning technique which performs clustering as well as dimensionality reduction. It can be used in combination with TF-IDF scheme to perform topic modeling.


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(extracted_text.values.astype('U'))

In [46]:
%%time

from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

CPU times: user 6min 4s, sys: 8.65 s, total: 6min 13s
Wall time: 38.6 s


NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [47]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

sullying
trousers
baloch
orroral
517
blanketed
vet
unifier
gotta
bearable


In [48]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [49]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

senate
republicans
plan
border
said
democrats
white
house
president
trump


In [50]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['senate', 'republicans', 'plan', 'border', 'said', 'democrats', 'white', 'house', 'president', 'trump']


Top 10 words for topic #1:
['pbs', 'com', 'click', 'headlines', 'daily', 'email', 'news', 'sign', 'breaking', 'subscribe']


Top 10 words for topic #2:
['use', 'sign', 'understand', 'agree', 'site', 'privacy', 'registration', 'user', 'constitutes', 'agreement']


Top 10 words for topic #3:
['customs', 'agency', 'arrests', 'arrested', 'agents', 'said', 'operation', 'enforcement', 'raids', 'ice']


Top 10 words for topic #4:
['topics', 'newsletter', 'news', 'national', 'local', 'sign', 'latest', 'breaking', 'delivered', 'inbox']


Top 10 words for topic #5:
['guatemala', 'southern', 'united', 'central', 'tariffs', 'mexican', 'asylum', 'migrants', 'border', 'mexico']


Top 10 words for topic #6:
['petitions', 'custom', 'competitors', 'profession', 'curve', 'motions', 'briefs', 'case', 'access', 'law360']


Top 10 words for topic #7:
['pbs', 'inbox', 'email'

### Metadata Description

Using this field to do topic modeling.


In [51]:
metadata_description=df_src_norm_limited['metadata.description']

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(metadata_description.values.astype('U'))

In [53]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [54]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

careerbuilder
ladders
colonization
fly
muddied
cite
setting
georgetown
june
kicks


In [55]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [56]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

campaign
hardline
said
tuesday
plan
washington
policies
donald
trump
president


In [57]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['campaign', 'hardline', 'said', 'tuesday', 'plan', 'washington', 'policies', 'donald', 'trump', 'president']


Top 10 words for topic #1:
['portland', 'director', 'county', 'custody', 'arrested', 'agents', 'agency', 'ice', 'customs', 'enforcement']


Top 10 words for topic #2:
['government', 'law', 'san', 'appeals', 'supreme', 'authorities', 'ruled', 'judge', 'court', 'federal']


Top 10 words for topic #3:
['ron', 'vitiello', 'donald', 'sweeps', 'chicago', 'washington', 'times', 'local', 'latest', 'ap']


Top 10 words for topic #4:
['crisis', 'policy', 'threatens', 'bringing', 'coordinate', 'shut', 'czar', 'patrol', 'southern', 'border']


Top 10 words for topic #5:
['workers', 'visas', 'based', 'international', 'plan', 'government', 'city', 'students', 'york', 'new']


Top 10 words for topic #6:
['deal', 'asylum', 'ap', 'city', 'tariffs', 'american', 'central', 'mexican', 'migrants', 'mexico']


Top 10 words for topic #7:
['liberty', 'agency', 'statue', 'u

### Keywords


In [58]:
metadata_keywords=df_src_norm_limited['metadata.keywords']

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(metadata_description.values.astype('U'))

In [60]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [61]:
# get random 10 words from the vocabulary

import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

permission
hatchet
libertarians
mixes
hhs
nod
preaches
patricia
straws
prosperous


In [62]:
#retrieve the probability vector of words for the first topic and will retrieve the indexes of the ten words with the highest probabilities

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]


In [63]:
#pull the actual words
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

campaign
hardline
said
tuesday
plan
washington
policies
donald
trump
president


In [64]:
#the ten words with highest probabilities for each of the topics:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['campaign', 'hardline', 'said', 'tuesday', 'plan', 'washington', 'policies', 'donald', 'trump', 'president']


Top 10 words for topic #1:
['portland', 'director', 'county', 'custody', 'arrested', 'agents', 'agency', 'ice', 'customs', 'enforcement']


Top 10 words for topic #2:
['government', 'law', 'san', 'appeals', 'supreme', 'authorities', 'ruled', 'judge', 'court', 'federal']


Top 10 words for topic #3:
['ron', 'vitiello', 'donald', 'sweeps', 'chicago', 'washington', 'times', 'local', 'latest', 'ap']


Top 10 words for topic #4:
['crisis', 'policy', 'threatens', 'bringing', 'coordinate', 'shut', 'czar', 'patrol', 'southern', 'border']


Top 10 words for topic #5:
['workers', 'visas', 'based', 'international', 'plan', 'government', 'city', 'students', 'york', 'new']


Top 10 words for topic #6:
['deal', 'asylum', 'ap', 'city', 'tariffs', 'american', 'central', 'mexican', 'migrants', 'mexico']


Top 10 words for topic #7:
['liberty', 'agency', 'statue', 'u